In [5]:
include("../src/SphereIFSCalib.jl")
using TwoDimensional, Zygote,StatsBase, Plots
plotly()

Plots.PlotlyBackend()

In [7]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
const λ4 = 1545.10e-9  # laser 4  
const λlaser = [λ1,λ2,λ3,λ4]
const λ0 = mean(λlaser);# reference


In [9]:
# model of the LensletModel
bbox = BoundingBox(xmin=1, ymin=1, xmax=10, ymax=50);
a0= rand(Float64,4);
fwhm0= rand(Float64,4)*10;
laser =  LaserModel(λlaser,a0,fwhm0)
lmod = LensletModel(λ0,laser.nλ-1,bbox);



In [11]:
Cinit = [[ 6.2 0 0 0 ]; [25 7e7 0 0]]
UpdateDispModel(lmod.dmodel,Cinit)
limage = LensletLaserImage(lmod,laser)
heatmap(limage)

<!DOCTYPE html>
 
 
 Plots.jl

In [12]:
function  likelihood(a::Array{Float64,1},fwhm::Array{Float64,1},C::Array{Float64,2}) 
    UpdateDispModel(lmod.dmodel, C);
    UpdateLaserModel(laser,a,fwhm);
    return sum((data .- LensletLaserImage(lmod,laser)).^2)
end

data= limage ;
cost = likelihood(a0,fwhm0,Cinit)


4.814943656993709

In [13]:
∇cost = gradient(likelihood,a0,fwhm0,Cinit)


([-1.9358189491863875, -2.010814804300717, -7.949412884990896, -4.290786946530049], [-0.19156291946793727, -1.7512215427550761, -2.2384495826291464, -0.34259891603791975], [0.24525886524845686 -3.2436856268489e-8 7.809995439887992e-15 -1.9560289702047287e-21; -0.21310359295726036 7.587701322460698e-8 -2.320976041886875e-14 7.360937417265843e-21])

In [14]:
∇cost = gradient(likelihood,a0,fwhm0 .+ 2.,Cinit)


([0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0])